In [7]:
import os
import shutil
import dill as pickle
import pandas as pd
import biogeme.biogeme as bio
from biogeme.models import loglogit
import pycmtensor as cmt
from pycmtensor.expressions import Beta

cmt.logger.set_level(cmt.logger.WARNING)
cmt.__version__


'0.6.1'

In [8]:
nb_path = os.path.abspath("")
model_average_sim = pd.read_csv("data/model_average_sim.csv")
model_average_sim.columns = model_average_sim.columns.str.upper()
# model_average_sp[["AV1", "AV2"]] = 1
db = cmt.Database("model_average_sim", model_average_sim, choiceVar="CHOICE")
globals().update(db.variables)

# additional steps to format database
db.data["CHOICE"] -= 1
time_columns = [
    "CAR_TIME",
    "AIR_TIME",
    "AIR_ACCESS",
    "RAIL_TIME",
    "RAIL_ACCESS",
    "HSR_TIME",
    "HSR_ACCESS",
]
cost_columns = ["CAR_COST", "AIR_COST", "RAIL_COST", "HSR_COST"]
income_columns = ["INCOME"]
db.autoscale(variables=time_columns, default=60.0, verbose=True)
db.autoscale(variables=cost_columns, default=100.0, verbose=True)
db.autoscale(variables=income_columns, default=1000.0, verbose=True)



In [9]:
# specify Beta parameters
b_cost = Beta("b_cost", 0.0, None, None, 0)
b_time = Beta("b_time", 0.0, None, None, 0)
b_access = Beta("b_access", 0.0, None, None, 0)
b_income_rail = Beta("b_income_rail", 0.0, None, None, 0)
b_income_hsr = Beta("b_income_hsr", 0.0, None, None, 0)
b_income_car = Beta("b_income_car", 0.0, None, None, 0)
b_female_car = Beta("b_female_car", 0.0, None, None, 0)
b_female_rail = Beta("b_female_rail", 0.0, None, None, 0)
b_female_air = Beta("b_female_air", 0.0, None, None, 0)

asc_car = Beta("asc_car", 0.0, None, None, 0)
asc_air = Beta("asc_air", 0.0, None, None, 0)
asc_rail = Beta("asc_rail", 0.0, None, None, 0)
asc_hsr = Beta("asc_hsr", 0.0, None, None, 1)

# specify weight parameters
# W1 = Weights("ResNet_01a", (4, 16), 0, True)
# W2 = Weights("ResNet_01b", (16, 4), 0, True)

U_car = (
    b_cost * CAR_COST
    + b_time * CAR_TIME
    + b_income_car * INCOME
    + b_female_car * FEMALE
    + asc_car
)
U_air = (
    b_cost * AIR_COST 
    + b_time * AIR_TIME 
    + b_access * AIR_ACCESS 
    + b_female_air * FEMALE
    + asc_air
)
U_rail = (
    b_cost * RAIL_COST
    + b_time * RAIL_TIME
    + b_access * RAIL_ACCESS
    + b_income_rail * INCOME
    + b_female_rail * FEMALE
    + asc_rail
)
U_hsr = (
    b_cost * HSR_COST
    + b_time * HSR_TIME
    + b_access * HSR_ACCESS
    + b_income_hsr * INCOME
    + asc_hsr
)

# Associate utility functions with the numbering of alternatives
V = {0: U_car, 1: U_air, 2: U_rail, 3: U_hsr}

# Associate the availability conditions with the alternatives
av = {0: 1, 1: 1, 2: 1, 3: 1}

# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
logprob = loglogit(V, av, CHOICE)

# Create the Biogeme object
biogeme = bio.BIOGEME(db, logprob)
biogeme.modelName = "logit_sim"

# Calculate the null log likelihood for reporting.
biogeme.calculateNullLoglikelihood(av)

if os.path.isdir(biogeme.modelName):
    shutil.rmtree(biogeme.modelName)
try:
    os.mkdir(biogeme.modelName)
    os.chdir(biogeme.modelName)

    # Estimate the parameters
    results = biogeme.estimate()
    os.chdir("..")
except:
    print("An exception occurred")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()

print(results)
print(pandasResults)



Results for model logit_sim
Output file (HTML):			logit_sim.html
Nbr of parameters:		12
Sample size:			4000
Excluded data:			0
Null log likelihood:		-5545.177
Init log likelihood:		-5545.177
Final log likelihood:		-3717.08
Likelihood ratio test (null):		3656.195
Rho square (null):			0.33
Rho bar square (null):			0.328
Likelihood ratio test (init):		3656.195
Rho square (init):			0.33
Rho bar square (init):			0.328
Akaike Information Criterion:	7458.16
Bayesian Information Criterion:	7533.688
Final gradient norm:		0.02045212
asc_air        : -1.17[0.216 -5.4 6.64e-08][0.212 -5.52 3.47e-08]
asc_car        : 0.792[0.195 4.07 4.77e-05][0.197 4.01 6e-05]
asc_rail       : -1.07[0.288 -3.72 0.000201][0.277 -3.86 0.000114]
b_access       : -0.425[0.106 -4 6.21e-05][0.106 -4.01 6.12e-05]
b_cost         : -2.98[0.115 -25.8 0][0.115 -25.8 0]
b_female_air   : 0.6[0.118 5.09 3.61e-07][0.116 5.15 2.6e-07]
b_female_car   : 0.18[0.0894 2.02 0.0437][0.0892 2.02 0.0433]
b_female_rail  : 0.449[0.159 2.83